In [ ]:
!pip install openai

In [140]:
import pandas as pd
import os
from openai import OpenAI

In [142]:
os.environ['OPENAI_API_KEY'] = 'sk-FkbJS3_PF98_3I0IBkOHhfjHagTwrPRsGDDde5nWO3T3BlbkFJG2NGzGeCtJ_lF9wjZEw77aF4sE9x7vsGh7RDqURV4A'

client = OpenAI()

In [143]:
def create_system_prompt():
    return "Generate a multiple-choice question on the specific topic. Include a list of possible answers and indicate the correct answer seperately. "

def create_test_prompt(topic, num_questions, num_options):
    return f"Topic: {topic}. For each of the {num_questions} multiple-choice questions, provide {num_options} options. After the options, type '[Correct Answer]:' followed by the correct option."

In [146]:
response = client.chat.completions.create(
    model ='gpt-3.5-turbo',
    messages = [
        {"role": "system", "content": "Generate a multiple_choice question on the specific topic.\
                                        Each question start with question number such as Question 1:\
                                        Include a list of possible answers and indicate the correct answer separately with '[Correct Answer]:'. \
                                        Format the question, option, and answer clearly."},
        {"role": "user", "content": create_test_prompt("Python Programming", 5, 4)},
    ]
)

In [147]:
response

ChatCompletion(id='chatcmpl-A9NMgNSa7StlRfuk776wUD3FazEgP', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Question 1: \nWhich data type is immutable in Python? \nA) List \nB) Dictionary \nC) Tuple \nD) Set \n[Correct Answer]: C) Tuple\n\nQuestion 2: \nWhat keyword is used to define a function in Python? \nA) var \nB) func \nC) def \nD) define \n[Correct Answer]: C) def\n\nQuestion 3: \nWhat is the output of the following code snippet: print(3 == 3.0)? \nA) True \nB) False \nC) Error \nD) None \n[Correct Answer]: A) True\n\nQuestion 4: \nWhich of the following is not a valid way to comment in Python? \nA) // This is a comment \nB) # This is a comment \nC) \'\'\' This is a comment \'\'\' \nD) """ This is a comment """ \n[Correct Answer]: A) // This is a comment\n\nQuestion 5: \nWhat does the \'len()\' function do in Python? \nA) Returns the last element of a list \nB) Returns the total number of characters in a string \nC) Returns th

In [148]:
print(response.choices[0].message.content)

Question 1: 
Which data type is immutable in Python? 
A) List 
B) Dictionary 
C) Tuple 
D) Set 
[Correct Answer]: C) Tuple

Question 2: 
What keyword is used to define a function in Python? 
A) var 
B) func 
C) def 
D) define 
[Correct Answer]: C) def

Question 3: 
What is the output of the following code snippet: print(3 == 3.0)? 
A) True 
B) False 
C) Error 
D) None 
[Correct Answer]: A) True

Question 4: 
Which of the following is not a valid way to comment in Python? 
A) // This is a comment 
B) # This is a comment 
C) ''' This is a comment ''' 
D) """ This is a comment """ 
[Correct Answer]: A) // This is a comment

Question 5: 
What does the 'len()' function do in Python? 
A) Returns the last element of a list 
B) Returns the total number of characters in a string 
C) Returns the length of an object 
D) Returns the maximum value in a list 
[Correct Answer]: C) Returns the length of an object


In [149]:
# separate an answer from question and save
def parse_student_view_and_answers(response):

    # initialize a dictionary
    student_view = {} # save some quetions and options that students see
    correct_answers = {} # save the answer from each question

    pattern = r"(Question \d+:) | (\\n\\n\[Correct Answer\]:)"
    question_blocks = re.split(pattern, response.choices[0].message.content)

    # Initialize the question number to 0. The reason I do this is to track the question number
    question_number = 0
    for block in question_blocks: #iterate through each block
        if not block or block.startswith("[Correct Answer]:"):
            continue
        if block.startswith("Question"): #If a block starts with a question, increase the question_number.
            question_number += 1
            continue

        # Separate question, option and answer from each block
        parts = block.split("[Correct Answer]:")
        question_with_options = parts[0].strip()
        correct_answer = parts[1].strip() if len(parts) > 1 else "Not provided"

        # save the student_view and the correct_answer in each dictionary 
        student_view[question_number] = question_with_options
        correct_answers[question_number] = correct_answer

    return student_view, correct_answers

In [150]:
import re
student_view, correct_answers = parse_student_view_and_answers(response)

In [151]:
student_view

{1: 'Which data type is immutable in Python? \nA) List \nB) Dictionary \nC) Tuple \nD) Set',
 2: 'What keyword is used to define a function in Python? \nA) var \nB) func \nC) def \nD) define',
 3: 'What is the output of the following code snippet: print(3 == 3.0)? \nA) True \nB) False \nC) Error \nD) None',
 4: 'Which of the following is not a valid way to comment in Python? \nA) // This is a comment \nB) # This is a comment \nC) \'\'\' This is a comment \'\'\' \nD) """ This is a comment """',
 5: "What does the 'len()' function do in Python? \nA) Returns the last element of a list \nB) Returns the total number of characters in a string \nC) Returns the length of an object \nD) Returns the maximum value in a list"}

In [152]:
correct_answers

{1: 'C) Tuple',
 2: 'C) def',
 3: 'A) True',
 4: 'A) // This is a comment',
 5: 'C) Returns the length of an object'}

## Test question excepted answer

In [154]:
for q_num, question_content in student_view.items():
    print(f"Question {q_num}:\n{question_content}\n")

Question 1:
Which data type is immutable in Python? 
A) List 
B) Dictionary 
C) Tuple 
D) Set

Question 2:
What keyword is used to define a function in Python? 
A) var 
B) func 
C) def 
D) define

Question 3:
What is the output of the following code snippet: print(3 == 3.0)? 
A) True 
B) False 
C) Error 
D) None

Question 4:
Which of the following is not a valid way to comment in Python? 
A) // This is a comment 
B) # This is a comment 
C) ''' This is a comment ''' 
D) """ This is a comment """

Question 5:
What does the 'len()' function do in Python? 
A) Returns the last element of a list 
B) Returns the total number of characters in a string 
C) Returns the length of an object 
D) Returns the maximum value in a list



## Simulation(from the student's point of view)

In [164]:
student_answers = {} # save the student's answer

for q_num, question_content in student_view.items():
    print(f"Question {q_num}:\n{question_content}\n")

    answer =input("Put your answer (example, A,B,C,D): ")
    student_answers[q_num] = answer
    print("\n")

Question 1:
Which data type is immutable in Python? 
A) List 
B) Dictionary 
C) Tuple 
D) Set



Put your answer (example, A,B,C,D):  A




Question 2:
What keyword is used to define a function in Python? 
A) var 
B) func 
C) def 
D) define



Put your answer (example, A,B,C,D):  C




Question 3:
What is the output of the following code snippet: print(3 == 3.0)? 
A) True 
B) False 
C) Error 
D) None



Put your answer (example, A,B,C,D):  D




Question 4:
Which of the following is not a valid way to comment in Python? 
A) // This is a comment 
B) # This is a comment 
C) ''' This is a comment ''' 
D) """ This is a comment """



Put your answer (example, A,B,C,D):  A




Question 5:
What does the 'len()' function do in Python? 
A) Returns the last element of a list 
B) Returns the total number of characters in a string 
C) Returns the length of an object 
D) Returns the maximum value in a list



Put your answer (example, A,B,C,D):  B


## Review the submmited answers

In [166]:
print("Review the submmited answers")

for q_num, question_content in student_view.items():
    print(f"Question {q_num}:\n{question_content}\n")
    print(f"Your answer: {student_answers[q_num]}\n")

print("---------------------------")

Review the submmited answers
Question 1:
Which data type is immutable in Python? 
A) List 
B) Dictionary 
C) Tuple 
D) Set

Your answer: A

Question 2:
What keyword is used to define a function in Python? 
A) var 
B) func 
C) def 
D) define

Your answer: C

Question 3:
What is the output of the following code snippet: print(3 == 3.0)? 
A) True 
B) False 
C) Error 
D) None

Your answer: D

Question 4:
Which of the following is not a valid way to comment in Python? 
A) // This is a comment 
B) # This is a comment 
C) ''' This is a comment ''' 
D) """ This is a comment """

Your answer: A

Question 5:
What does the 'len()' function do in Python? 
A) Returns the last element of a list 
B) Returns the total number of characters in a string 
C) Returns the length of an object 
D) Returns the maximum value in a list

Your answer: B

---------------------------


In [168]:
correct_answers

{1: 'C) Tuple',
 2: 'C) def',
 3: 'A) True',
 4: 'A) // This is a comment',
 5: 'C) Returns the length of an object'}

In [170]:
student_answers[1][0]

'A'

In [172]:
correct_answers.items()

dict_items([(1, 'C) Tuple'), (2, 'C) def'), (3, 'A) True'), (4, 'A) // This is a comment'), (5, 'C) Returns the length of an object')])

In [179]:
score = 0

for q_num, correct_answer in correct_answers.items():
    correct_option_letter = correct_answer[0]

    if student_answers[q_num] == correct_option_letter:
        score += 1

total_questions = len(student_view)
percentage_score = (score / total_questions) * 100

print(f"Your score is: {score}/{total_questions} ({percentage_score}%) ")

if percentage_score >= 60:
    print("Congratulation. You passed the exam")
else:
    print("You failed the exam. You need to retake the exam.")

Your score is: 2/5 (40.0%) 
You failed the exam. You need to retake the exam.


## modularization

In [196]:
def collect_student_answers(student_view):
    student_answers = {}  # 딕셔너리 초기화
    for q_num, question_content in student_view.items(): 
        print(f"Question {q_num}:\n{question_content}\n")
        answer = input("Put your answer (example, A,B,C,D): ")
        student_answers[q_num] = answer
        print("\n")
    return student_answers

def review_student_answers(student_view, student_answers):
    print("Review the submmited answers", "\n")

    for q_num, question_content in student_view.items():
        print(f"Question {q_num}:\n{question_content}\n")
        print(f"Your answer: {student_answers[q_num]}\n")

def grade_exam(correct_answers, student_answers):
    score = 0
    
    for q_num, correct_answer in correct_answers.items():
        correct_option_letter = correct_answer[0]
    
        if student_answers[q_num] == correct_option_letter:
            score += 1
    
    total_questions = len(student_view)
    percentage_score = (score / total_questions) * 100
    print(f"Your score is: {score}/{total_questions} ({percentage_score}%) ")

    pass_threshold = 60
    if percentage_score >= pass_threshold:
        print("Congratulation. You passed the exam")
    else:
        print("You failed the exam. You need to retake the exam.")

In [198]:
student_answers = collect_student_answers(student_view)

review_student_answers(student_view, student_answers)

grade_exam(correct_answers, student_answers)

Question 1:
Which data type is immutable in Python? 
A) List 
B) Dictionary 
C) Tuple 
D) Set



Put your answer (example, A,B,C,D):  C




Question 2:
What keyword is used to define a function in Python? 
A) var 
B) func 
C) def 
D) define



Put your answer (example, A,B,C,D):  A




Question 3:
What is the output of the following code snippet: print(3 == 3.0)? 
A) True 
B) False 
C) Error 
D) None



...



Question 4:
Which of the following is not a valid way to comment in Python? 
A) // This is a comment 
B) # This is a comment 
C) ''' This is a comment ''' 
D) """ This is a comment """



Put your answer (example, A,B,C,D):  D




Question 5:
What does the 'len()' function do in Python? 
A) Returns the last element of a list 
B) Returns the total number of characters in a string 
C) Returns the length of an object 
D) Returns the maximum value in a list



Put your answer (example, A,B,C,D):  B




Review the submmited answers 

Question 1:
Which data type is immutable in Python? 
A) List 
B) Dictionary 
C) Tuple 
D) Set

Your answer: C

Question 2:
What keyword is used to define a function in Python? 
A) var 
B) func 
C) def 
D) define

Your answer: A

Question 3:
What is the output of the following code snippet: print(3 == 3.0)? 
A) True 
B) False 
C) Error 
D) None

Your answer: C

Question 4:
Which of the following is not a valid way to comment in Python? 
A) // This is a comment 
B) # This is a comment 
C) ''' This is a comment ''' 
D) """ This is a comment """

Your answer: D

Question 5:
What does the 'len()' function do in Python? 
A) Returns the last element of a list 
B) Returns the total number of characters in a string 
C) Returns the length of an object 
D) Returns the maximum value in a list

Your answer: B

Your score is: 1/5 (20.0%) 
You failed the exam. You need to retake the exam.
